In [174]:
import numpy as np
import cv2
import os
import pandas as pd
from skimage import io, transform, color, measure, segmentation, morphology, feature
from sklearn import svm, multiclass, model_selection
import csv
import matplotlib.pyplot as plt

LBP 的特征提取使用skimage里定义好的库

In [175]:
# fullfil the space
def imfill(im_th):
#     print('im_th', im_th)
    # im_th 是0 1 整形二值图
    # Copy the thresholded imapge.
    im_th = np.uint8(im_th)
    im_floodfill = im_th.copy()
    # Mask used to flood filling.
    # Notice the size needs to be 2 pixels than the image.
    h, w = im_th.shape[:2]
    # print('h '+str(h)+'   w '+str(w))
    mask = np.zeros((h + 2, w + 2), np.uint8)
    # Floodfill from point (0, 0)
    cv2.floodFill(im_floodfill, mask, (0, 0), 1)
    # Invert floodfilled image
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)
    # Combine the two images to get the foreground.
    im_out = im_th | im_floodfill_inv
#     print('im_out', im_out)
    return im_out

In [184]:
path_dir = 'E:/pycharm_project/seg_region/pos.csv'

In [185]:
data = pd.read_csv(path_dir, sep = "\s+|\t+|\s+\t+|\t+\s+")

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [186]:
data.columns

Index(['path', 'label'], dtype='object')

In [187]:
imgs = data['path'].values

In [188]:
label = data['label'].values
len(label)

3129

In [192]:
Data = []
for img in imgs:
    img = cv2.imread(img)
    gray = color.rgb2gray(img)
#     print(gray<100/255) here will print true of false
#     set the threshold = 100
    bw = (gray<100/255)*1
    
    img_fill=imfill(bw)
    copy_img = img_fill.copy()
    # 清除与边界相连的目标物
    segmentation.clear_border(copy_img)
    # 连通区域标记 
    label_img= measure.label(copy_img)
    
#     remove the background
    borders = np.logical_xor(bw, copy_img)
    label_img[borders] = -1
#   离心率  
    eccentricity = 1  
    
# 循环得到每一个连通区域属性集
            # 忽略小区域
    for region in measure.regionprops(label_img):
#         print(region.area)
        if region.area<2000:
            continue
        
#       draw a outbound rectangular      
        if eccentricity>region.eccentricity:
#             print(region.eccentricity)
            eccentricity=region.eccentricity
            minr, minc, maxr, maxc = region.bbox
#     print(vars())
#  判断是否有符合条件的区域。
    if 'minr' in vars():
        pic = img[minr:maxr, minc: maxc, :]
#         plt.imshow(pic)
#         plt.show()

        pic = transform.resize(pic, [30,30,3])
    else:
        pic = transform.resize(img, [30,30,3])
        
        
    pic_gray = color.rgb2gray(pic)
    rows, cols = pic_gray.shape

    radius = 2
    n_points = radius*8

    lbp_sum = []
    for row in range(2):
        for col in range(3):
            pic_block = pic_gray[(row*rows//2):((row+1)*rows//2-1),
                                 (col*col//2):((col+1)*col//2-1)]
#             print(pic_block)
            lbp = feature.local_binary_pattern(pic_gray, n_points, 
                                               radius, 'uniform')

            lbp_ = lbp.astype(np.int32)
            max_bins = int(lbp_.max()+1)
            train_hist, _ = np.histogram(lbp_, normed=True, bins=max_bins, 
                                         range=(0, max_bins))

            lbp_sum= lbp_sum+train_hist.tolist()
    Data.append(lbp_sum)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.


SVM training process 

In [193]:
len(Data)

3129

In [194]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(Data, label, 
                                                                    test_size=0.2,
                                                                   random_state = 0)

In [195]:
svm_svr = svm.SVC(kernel='rbf', C=15, gamma='scale')
model = multiclass.OneVsRestClassifier(svm_svr, -1)

In [196]:
clf = model.fit(X_train, y_train)
score = clf.score(X_test, y_test)
print(score)

0.9281150159744409
